In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR='D:\Temp\MachineLearning\data\cifar-10-batches-py'

class   CifarData:
    def __init__(self,filelist,shuffle):
        all_data=[]
        all_labels=[]
        #循环读取多个批次的数据集文件
        for filename in filelist:
            data,labels=self.load_data(filename)      
            all_data.append(data)
            all_labels.append(labels)
        # 将所有data以纵向的方式合并到_data
        self._data=np.vstack(all_data)
        # 数据归一化处理
        self._data=self._data/127.5 - 1
        # 将所有label以横向方式合并到_label
        self._labels=np.hstack(all_labels)
        # 数据中样本个数
        self._example_num=self._data.shape[0]
        print(self._example_num)
        # 定位数据集遍历的位置指针
        self._indicator=0
        # 根据传入的shuffle决定是否对数据进行打乱
        self._shuffle=shuffle
        if self._shuffle:
            self.shuffle_data()
                 
    def load_data(self,filename):
        """读取一个批文件中的数据"""
        with open(filename,'rb') as f:
            data=pickle.load(f,encoding='bytes')
            return data[b'data'],data[b'labels']
        
    def shuffle_data(self):
        # 返回一个打乱的序号列表，例如将0~5 随机打乱为 [5,3,2,4,0,1]
        shuffle_list=np.random.permutation(self._example_num)
        # 根据随机列表重排数据和对应的标签值
        self._data=self._data[shuffle_list]
        self._labels=self._labels[shuffle_list]
        
    def next_batch(self,batch_size ):
        '''返回当前指针self._indicator之后batch_size个数据'''
        end_indicator=self._indicator+batch_size
        # 如果相加结果超过了样本个数
        if end_indicator>self._example_num:
            # 如果可以洗牌，打乱数据并从0开始重新取数据
            if self._shuffle:
                self.shuffle_data()
                self._indicator=0
                end_indicator=batch_size
            else:
                raise Exception("没有更多数据了！")    
        # 返回从_indicator到end_indicator的data和labels，并将指针后移到end_indicator
        batch_data=self._data[self._indicator:end_indicator]
        batch_labels=self._labels[self._indicator:end_indicator]
        self._indicator=end_indicator
        return batch_data,batch_labels

# 拼接训练数据的路径名        
train_file=[os.path.join(CIFAR_DIR,'data_batch_%d'%i) for i in range (1,6)]
test_file=[os.path.join(CIFAR_DIR,'test_batch')]
# 创建训练数据对象
train_data=CifarData(train_file,True)     


def inception_block(x,out_chanel_list,name):
    '''
    实现 inception网络块
    :param x: 输入
    :param out_chanel_list: 不同卷积核输出通道列表
    :param name: 命名域名称
    :return: 多个卷积层的拼接结果
    '''
    with tf.variable_scope(name):
        # 定义1×1、3×3、5×5与max_pooling层
        conv1_1=tf.layers.conv2d(x,out_chanel_list[0],(1,1),(1,1),
                                 padding='same',activation=tf.nn.relu,name='conv1_1')
        conv3_3=tf.layers.conv2d(x,out_chanel_list[1],(3,3),(1,1),
                                 padding='same',activation=tf.nn.relu,name='conv3_3')
        conv5_5=tf.layers.conv2d(x,out_chanel_list[2],(5,5),(1,1),
                                 padding='same',activation=tf.nn.relu,name='conv5_5')
        max_pooling=tf.layers.max_pooling2d(x,(2,2),(2,2),name='max_pooling')
    # 对max_pooling的结果进行填充，将mx与input相差的部分填充到mx的两侧
    mx_shape=max_pooling.shape[1:]
    input_shape=x.shape[1:]
    width_padding=(input_shape[0]-mx_shape[0])//2
    height_padding=(input_shape[1]-mx_shape[1])//2
    # max_pooling->[None,width,height,chanel]，对中间的宽和高进行填充
    mx_padded=tf.pad(max_pooling,[[0,0],[width_padding,width_padding],[height_padding,height_padding],[0,0]])
    # 将四个输出结果在第三个维度上进行拼接
    concat_layer=tf.concat([conv1_1,conv3_3,conv5_5,mx_padded],axis=3)   
    return concat_layer

x=tf.placeholder(tf.float32,[None,3072])
y=tf.placeholder(tf.int64,[None])
# 将一维向量x转化为具有三通道的多维图片，并交换向量通道
x_img=tf.reshape(x,[-1,3,32,32])
x_img=tf.transpose(x_img,[0,2,3,1])

# 先经过一个卷积层和池化层
conv1=tf.layers.conv2d(x_img,32,(3,3),padding='same',activation=tf.nn.relu,name='conv1')
pooling1=tf.layers.max_pooling2d(conv1,(2,2),(2,2),name='pooling1')   
# 第二层经过两个inception块和池化层
inception_2a=inception_block(pooling1,[16,16,16],'inception_2a')
inception_2b=inception_block(inception_2a,[16,16,16],'inception_2b')
pooling2=tf.layers.max_pooling2d(inception_2b,(2,2),(2,2),name='pooling2')
# 第三层
inception_3a=inception_block(pooling2,[16,16,16],'inception_3a')
inception_3b=inception_block(inception_3a,[16,16,16],'inception_3b')
pooling3=tf.layers.max_pooling2d(inception_3b,(2,2),(2,2),name='pooling3')

# 将结果展开，进行全连接层
flatten=tf.layers.flatten(pooling3)
y_predict=tf.layers.dense(flatten,10)
# 使用交叉熵作为损失函数
loss=tf.losses.sparse_softmax_cross_entropy(y,y_predict)

# 将标签预测向量中最大的下标作为预测值，例如[0.1,0.8...0.01]，则预测为第二类
predict=tf.math.argmax(y_predict,1)
# 通过equal函数逐一比较predict，y_reshape的每一个元素
correction=tf.equal(predict,y)
accuracy=tf.reduce_mean(tf.cast(correction,tf.float64))
#定义优化方法
with tf.name_scope('train_op'):
    train_op=tf.train.AdamOptimizer(1e-3).minimize(loss)
    
init=tf.global_variables_initializer()
batch_size=20
train_steps=5000
test_steps=100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data,batch_labels=train_data.next_batch(batch_size)
        loss_val,acc_val,_=sess.run([loss,accuracy,train_op],feed_dict={x:batch_data,y:batch_labels})
        if (i+1) % 500==0:
            print("第%d步：损失：%.5f，精确度：%.5f"%(i,loss_val,acc_val))
        #每训练1000次，在test数据集上进行一次测试
        if (i+1)%1000==0:
            # 定义测试集数据对象
            test_data=CifarData(test_file,False)   
            all_acc=[]
            for j in range(test_steps):
                test_batch_data,test_batch_labels=test_data.next_batch(batch_size)
                test_acc=sess.run([accuracy],feed_dict={x:test_batch_data,y:test_batch_labels})
                all_acc.append(test_acc)
            # 将测得的多个准确率求均值
            test_acc_mean=np.mean(all_acc)
            print("第%d步测试集准确率%.4f"%(i,test_acc_mean))
            
            

D:\Program\IDE\Anaconda\envs\TF-GPU\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Program\IDE\Anaconda\envs\TF-GPU\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Program\IDE\Anaconda\envs\TF-GPU\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Program\IDE\Anaconda\envs\TF-GPU\lib\site-packages\tensorflow\python\f

50000
第499步：损失：1.56344，精确度：0.45000
第999步：损失：1.38347，精确度：0.55000
10000
第999步测试集准确率0.5610
第1499步：损失：0.88997，精确度：0.65000
第1999步：损失：1.64298，精确度：0.45000
10000
第1999步测试集准确率0.6120
第2499步：损失：0.76972，精确度：0.80000
第2999步：损失：0.69439，精确度：0.75000
10000
第2999步测试集准确率0.6795
第3499步：损失：1.25360，精确度：0.60000
第3999步：损失：0.74434，精确度：0.70000
10000
第3999步测试集准确率0.6860
第4499步：损失：0.72573，精确度：0.80000
第4999步：损失：0.84710，精确度：0.70000
10000
第4999步测试集准确率0.7070
